<a href="https://colab.research.google.com/github/kshitizs11/Image-To-Text-Convertor-Using-Ocr-/blob/main/OCR(Practice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz"

--2020-09-07 08:10:58--  https://www.robots.ox.ac.uk/~vgg/data/text/mjsynth.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10678411583 (9.9G) [application/x-gzip]
Saving to: ‘mjsynth.tar.gz’

mjsynth.tar.gz      100%[===================>]   9.94G  11.7MB/s    in 14m 44s 

2020-09-07 08:25:44 (11.5 MB/s) - ‘mjsynth.tar.gz’ saved [10678411583/10678411583]



In [1]:
import tarfile
with tarfile.open('mjsynth.tar.gz') as tar:
    tar.extractall()
# 22.60 or a bit more than it

In [ ]:
from keras.preprocessing import image
import os
import numpy as np
import cv2

In [ ]:
path = '/mnt/ramdisk/max/90kDICT32px'
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []

max_label_len = 0 
i=1
flag =0

In [ ]:
import string

In [ ]:
char_list = string.ascii_letters + string.digits
char_list.index("abcdefgh")
def encode_to_labels(txt):
  dig_lst = []
  for index,char in enumerate(txt):
    # print(index,char)
    try:
      dig_lst.append(char_list.index(char))
    except:
      print(char)
  return dig_lst

In [ ]:
encode_to_labels("abcdefgh")

[0, 1, 2, 3, 4, 5, 6, 7]

# Image Extraction

In [ ]:

for folders in os.listdir('mnt/ramdisk/max/90kDICT32px'):
  if folders.split(".")[-1]=="txt":
    txt_file.append(folders)
    continue
  for num in os.listdir('mnt/ramdisk/max/90kDICT32px/'+folders):
    for images in os.listdir("mnt/ramdisk/max/90kDICT32px/"+folders+"/"+num):
      # print(images)
      txt = images.split("_")[1]
      if len(txt)>max_label_len:
        max_label_len = len(txt)
      if images.split(".")[1]=="jpg":
        
        im = cv2.cvtColor(cv2.imread(os.path.join("mnt/ramdisk/max/90kDICT32px/"+folders+"/"+num+"/",images)),cv2.COLOR_BGR2GRAY)
        w,h = im.shape
        if h>128 or w>32:
          continue
        if w<32:
          add_zeros = np.ones((32-w,h))*255
          im = np.concatenate((im, add_zeros))
        if h<128:
          add_zeros = np.ones((32,128-h))*255
          im = np.concatenate((im, add_zeros), axis=1)
        im = np.expand_dims(im,axis=2)
        im=im/255
        # print(images)
        # plt.imshow(im.reshape(32,128),cmap="gray")
        # plt.show()
        # print(im.shape)
        if i%10==0:
          valid_img.append(im)
          # valid_txt.append()
          valid_orig_txt.append(txt)
          valid_label_length.append(len(txt))
          valid_input_length.append(31)
          valid_txt.append(encode_to_labels(txt))
        else:
          training_img.append(im)
          orig_txt.append(txt)   
          train_label_length.append(len(txt))
          train_input_length.append(31)
          training_txt.append(encode_to_labels(txt))
        if i==100000:
          flag=1
          break
    
        i+=1

    if flag==1:
      break
  if flag==1:
    break

          # training_txt.append(encode_to_labels(txt))
        # x.append(im)

# Image

In [ ]:
val_img = np.array(valid_img)
train_img = np.array(training_img)
val_img.shape,train_img.shape

((10000, 32, 128, 1), (90000, 32, 128, 1))

In [ ]:
orig_txt = np.array(orig_txt)
valid_orig_txt = np.array(valid_orig_txt)
orig_txt = orig_txt[:90000]
valid_orig_txt = valid_orig_txt[:10000]
orig_txt.shape,valid_orig_txt.shape

((90000,), (10000,))

In [ ]:
# for i in range(len(val_img)):
#   print(valid_orig_txt[i])
#   plt.imshow(val_img[i].reshape(32,128),cmap="gray")
#   plt.show()

In [ ]:
train_input_length = np.array(train_input_length)
valid_input_length = np.array(valid_input_length)
train_input_length = train_input_length[:90000]
valid_input_length = valid_input_length[:10000]
train_input_length.shape,valid_input_length.shape

((90000,), (10000,))

In [ ]:
valid_label_length = np.array(valid_label_length)
train_label_length = np.array(train_label_length)
valid_label_length = valid_label_length[:10000]
train_label_length = train_label_length[:90000]
train_label_length.shape,valid_label_length.shape

((90000,), (10000,))

In [ ]:
training_txt = np.array(training_txt)
valid_txt = np.array(valid_txt)
training_txt = training_txt[:90000]
valid_txt = valid_txt[:10000]
training_txt.shape,valid_txt.shape

((90000,), (10000,))

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
training_txt_pad =  pad_sequences(training_txt,maxlen = max_label_len,padding="post",value = len(char_list))
val_txt_pad = pad_sequences(valid_txt,maxlen = max_label_len,padding="post",value = len(char_list))

In [ ]:
training_txt_pad.shape,val_txt_pad.shape

((90000, 23), (10000, 23))

In [ ]:

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [ ]:
# K?

In [ ]:
inputs = Input(shape=(32,128,1))

# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64,(3,3),activation="relu",padding="same")(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2,2),strides=2)(conv_1)

conv_2 = Conv2D(128,(3,3),activation="relu",padding="same")(pool_1)
pool_2 = MaxPool2D(pool_size=(2,2),strides=2)(conv_2)

conv_3 = Conv2D(256,(3,3),activation="relu",padding="same")(pool_2)


# conv_4 = Conv2D(256,(3,3),activation="relu",padding="same")(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_3)

conv_5 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 

conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
# 512 ->Layer thi niche
conv_7 = Conv2D(256, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(64, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(64, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)
 
act_model = Model(inputs, outputs)

In [ ]:
act_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 128, 1)]      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 128, 64)       640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 64, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 32, 128)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 32, 256)        295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 32, 256)       

# CTC LOSS Function Model For Training Purpose

In [ ]:
labels = Input(name="the_labels",shape=[max_label_len],dtype="float32")
input_length = Input(name="input_length",shape=[1],dtype="int64")
label_length = Input(name="label_length",shape=[1],dtype="int64")

def ctc_lambda_func(args):
  
  y_pred,labels,input_length,label_length = args
  
  return K.ctc_batch_cost(labels,y_pred,input_length,label_length)
loss_out = Lambda(ctc_lambda_func,output_shape=(1,),name="ctc")([outputs, labels, input_length, label_length])
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 128, 1)] 0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 32, 128, 64)  640         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 16, 64, 64)   0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 16, 64, 128)  73856       max_pooling2d_3[0][0]            
_______________________________________________________________________________________

# Model Compile

In [ ]:
model.compile(loss={"ctc":lambda y_true,y_pred:y_pred},optimizer="adam")

#Filepath
file_path = "best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=file_path,monitor="val_loss",verbose=1,save_best_only=True,mode="auto")
callbacks_list = [checkpoint]

# Fitting The Model

In [ ]:
model.fit(x=[train_img, training_txt_pad, train_input_length, train_label_length]
          , y=np.zeros(len(train_img)), batch_size=256, epochs = 10,
          validation_data = ([val_img, val_txt_pad, valid_input_length,
                              valid_label_length],
          [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

Epoch 1/10
352/352 [==============================] - ETA: 0s - loss: 28.2295
Epoch 00001: val_loss improved from inf to 27.17811, saving model to best_model.hdf5
352/352 [==============================] - 148s 422ms/step - loss: 28.2295 - val_loss: 27.1781
Epoch 2/10
352/352 [==============================] - ETA: 0s - loss: 25.5237
Epoch 00002: val_loss improved from 27.17811 to 25.02816, saving model to best_model.hdf5
352/352 [==============================] - 143s 405ms/step - loss: 25.5237 - val_loss: 25.0282
Epoch 3/10
352/352 [==============================] - ETA: 0s - loss: 22.4170
Epoch 00003: val_loss improved from 25.02816 to 20.35130, saving model to best_model.hdf5
352/352 [==============================] - 143s 406ms/step - loss: 22.4170 - val_loss: 20.3513
Epoch 4/10
352/352 [==============================] - ETA: 0s - loss: 12.9746
Epoch 00004: val_loss improved from 20.35130 to 8.17379, saving model to best_model.hdf5
352/352 [==============================] - 143s 4

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# type(val_img)

numpy.ndarray

In [ ]:
# load the saved best model weights
act_model.load_weights("best_model.hdf5")


# predict outputs on validation images
prediction = act_model.predict(val_img)

#use CTC decodeer
out = K.get_value(K.ctc_decode(prediction,input_length=np.ones(prediction.shape[0])*prediction.shape[1],greedy=True))[0][0]

# see The result
i=0
for x in out:
  print(valid_orig_txt[i])
  # plt.imshow(val_img[i].reshape(32,128),cmap="gray")
  # plt.show()
  for p in x:
    if int(p)!=-1:
      print(char_list[int(p)],end="")
  print("\n")
  i+=1

Streaming output truncated to the last 5000 lines.

Caspian
Casplan

FIREFIGHTER
FRTCNES

Vegetating
vegefating

telemarketer
telemaketer

Effusive
Erfusve

exegetical
cregered

Shitfaced
Shitfaced

Inauspiciously
haupiciously

Sidecars
Sidecars

LOLL
LOLL

RENAMES
RENAMES

STARGAZERS
STARGAZERS

wire
Wire

LUSHLY
LUSHLY

Astronaut
Astronaut

Monet
Monet

Mishears
MISHEARS

tatamis
tatamis

exemptions
exemptions

FEW
FEW

Likableness
Likableness

SALESLADY
SALESLAY

Pomona
Pomona

MISOGAMISTS
MISOGAMISTS

DRYADS
DRYADS

ORIGINATING
DHBINATING

Intent
Intent

Army
Army

Subpoenaing
Subpoenaing

Invigilate
Invigilate

nymph
Nymph

ADJOINED
ADJOINED

misdirecting
misdirecting

hying
hying

wimp
wimp

Goosesteps
Goosesteps

lynchers
ynchers

coroners
corbpers

bakery
bakery

TRAVAILS
TRAVAILS

PICCADILLY
PICCADILLY

CUP
CUP

Woodcarvers
Wootcenvcrs

jazz
juzz

MELANCHOLIA
MELANCHOLIA

likewise
Likewse

eerier
eerier

African
AFRICaN

Gawain
snin

Amaze
Amaze

JAIME
Jaime

oxidizers
oxidige

(13500,)

(1500,)

(13500,)

(1500,)